# Formattazione dei dataset ISTAT
Ri-formatto i dataset ISTAT con procedura di cleaning. Una volta creati i vari dataset, verranno mergiati in un unico dataset a cui si procederà al feature engineering

In [1]:
# Librerie e costanti da usare
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

from tqdm import tqdm
from utils.constant import *

### Dataset dagli anni 1982 agli anni 1991

In [2]:
# Lettura dei dati

SOURCE_PATH = PATH_RAW_DATA + '1982-1991/'

datasets = []

for filename in os.listdir(SOURCE_PATH):
    if filename.endswith('.csv'):
        datasets.append(pd.read_csv(SOURCE_PATH + filename, sep=';', encoding='latin1'))

In [3]:
# Pulizia dei dati

for i, dataset in enumerate(datasets):
    dataset = dataset[dataset["Tipo aggregazione"].isin(["Regione", "Totale"])]
    dataset = dataset[dataset["Sesso"] != "T"]
    dataset = dataset[dataset["ETA"] != 99]
    
    dataset = dataset.drop(columns=["Tipo aggregazione", "Codice aggregazione"])
    dataset["Aggregazione"] = dataset["Aggregazione"].str.capitalize()
    datasets[i] = dataset

In [4]:
# Strutturazione dei dati

COLUMN_NAMES = {
    "Aggregazione": "Area",
    "Sesso": "Sesso",
    "ETA": "Eta",
    "Anno": "Anno"
}

def categorize_age(age):
    for i, age_range in enumerate(AGE_GROUP["categories"]):
        if age in age_range:
            return AGE_GROUP["age_labels"][i]
    else:
        return AGE_GROUP["age_labels"][-1]
    
    

for i, dataset in enumerate(datasets):
    dataset = dataset.melt(id_vars=[ "Aggregazione", "Sesso", "ETA"],
                        var_name="Anno", value_name="Popolazione")
    
    dataset['Anno'] = dataset['Anno'].str.extract('(\d+)').astype(int)
    
    dataset['Fascia_Eta'] = pd.cut(dataset['ETA'], bins=AGE_GROUP["age_bins"], labels=AGE_GROUP["age_labels"], right=True, include_lowest=True)

    dataset_pivot = dataset.pivot_table(index=['Aggregazione', 'Anno', 'Sesso'], 
                                        columns='Fascia_Eta', 
                                        values='Popolazione', 
                                        aggfunc='sum',
                                        observed=True).reset_index()
    dataset_pivot.columns.name = None 
    dataset_pivot = dataset_pivot.rename_axis(None, axis=1) 
    dataset_pivot = dataset_pivot.rename(columns=COLUMN_NAMES)
    datasets[i] = dataset_pivot

In [5]:
# Salvataggio dei dati

DESTINATION_PATH = PATH_CLEANED_DATA

if not os.path.exists(DESTINATION_PATH):
    os.makedirs(DESTINATION_PATH)
    
final_dataset = pd.concat(datasets, ignore_index=True)
final_dataset.to_parquet(DESTINATION_PATH + 'popolazione_1982-1991.parquet', index=False)
final_dataset.to_csv(DESTINATION_PATH + 'popolazione_1982-1991.csv', index=False)

### Dataset dagli anni 1992 agli anni 2001

In [6]:
# Lettura dei dati

SOURCE_PATH = PATH_RAW_DATA + '1992-2001/'

datasets = {}

for filename in os.listdir(SOURCE_PATH):
    if filename.endswith('.csv'):
        year = filename.split('.')[0]
        datasets[year] = pd.read_csv(SOURCE_PATH + filename, encoding='latin1')

In [7]:
# Pulizia dei dati

for k, df in datasets.items():
    df = df[df["Sesso"] != "Totale"]
    df = df.drop(columns=["Codice regione"])
    datasets[k] = df

In [8]:
# Strutturazione dei dati

COLUMN_NAMES = {
    "Regione": "Area",
    "Sesso": "Sesso",
    "ETA": "Eta",
    "Anno": "Anno"
}

for k, df in datasets.items():
    df["Anno"] = int(k)
    df["Sesso"] = df["Sesso"].replace({"Maschi": "M", "Femmine": "F", "Totale": "T"})
    df["ETA"] = df["ETA"].replace({"100 e oltre": 100}).astype(int)
    df["Popolazione"] = df["Popolazione"].astype(int)
    
    
    df['Fascia_Eta'] = pd.cut(df['ETA'], bins=AGE_GROUP["age_bins"], labels=AGE_GROUP["age_labels"], right=True, include_lowest=True)

    dataset_pivot = df.pivot_table(index=['Regione', 'Anno', 'Sesso'], 
                                        columns='Fascia_Eta', 
                                        values='Popolazione', 
                                        aggfunc='sum',
                                        observed=True).reset_index()

    dataset_pivot.columns.name = None 
    dataset_pivot = dataset_pivot.rename_axis(None, axis=1) 
    dataset_pivot = dataset_pivot.rename(columns=COLUMN_NAMES)
    datasets[k] = dataset_pivot
    

> Visto che il dataset di questo periodo non presentano come Area l'Italia, dobbiamo applicare tecniche di feature engineering per crearla artifecialmente. \
Semplicemente sommiamo di uno specifico anno e sesso per ogni regione e creamo i dati relativi al paese.

In [9]:
# Aggiunta del terrorio nazionale


for k, df in datasets.items():
    df: pd.DataFrame
    years = list(df["Anno"].unique())
    
    for year in years:
        df_year = df[df["Anno"] == year]
        
        # Creazione del territorio nazionale Maschile
        df_year_m = df_year[df_year["Sesso"] == "M"]
        new_item_m = {
            "Area": "Italia",
            "Anno": year,
            "Sesso": "M",
            "0-9": df_year_m["0-9"].sum(),
            "10-19": df_year_m["10-19"].sum(),
            "20-29": df_year_m["20-29"].sum(),
            "30-39": df_year_m["30-39"].sum(),
            "40-49": df_year_m["40-49"].sum(),
            "50-59": df_year_m["50-59"].sum(),
            "60-69": df_year_m["60-69"].sum(),
            "70-79": df_year_m["70-79"].sum(),
            "80+": df_year_m["80+"].sum(),
        }
        new_df_m = pd.DataFrame([new_item_m])
        
        # Creazione del territorio nazionale Femminile
        df_year_f = df_year[df_year["Sesso"] == "F"]
        new_item_f = {
            "Area": "Italia",
            "Anno": year,
            "Sesso": "F",
            "0-9": df_year_f["0-9"].sum(),
            "10-19": df_year_f["10-19"].sum(),
            "20-29": df_year_f["20-29"].sum(),
            "30-39": df_year_f["30-39"].sum(),
            "40-49": df_year_f["40-49"].sum(),
            "50-59": df_year_f["50-59"].sum(),
            "60-69": df_year_f["60-69"].sum(),
            "70-79": df_year_f["70-79"].sum(),
            "80+": df_year_f["80+"].sum(),
        }
        new_item_f = pd.DataFrame([new_item_f])
        
        update_df = pd.concat([df, new_df_m, new_item_f], ignore_index=True)
        datasets[k] = update_df

In [10]:
# Salvataggio dei dati

DESTINATION_PATH = PATH_CLEANED_DATA

if not os.path.exists(DESTINATION_PATH):
    os.makedirs(DESTINATION_PATH)
    
final_dataset = pd.concat(datasets, ignore_index=True)

final_dataset = final_dataset.sort_values(by=["Area", "Anno", "Sesso"])
final_dataset.to_parquet(DESTINATION_PATH + 'popolazione_1992-2001.parquet', index=False)
final_dataset.to_csv(DESTINATION_PATH + 'popolazione_1992-2001.csv', index=False)

### Dataset dagli anni 2002 agli anni 2018

Il file CSV di questo dataset è formattato in modo non comune e ciò complica la lettura da parte della libreria pandas. \
Per risolvere questo problema, creamo un codice che dal fi

In [11]:
# Formatazione dei file CSV
SOURCE_PATH = PATH_RAW_DATA + '2002-2018/'

DESTINATION_PATH = SOURCE_PATH + 'formatted/'

BLACKLIST = [
    "\"Ricostruzione della popolazione intercensuaria - Popolazione al 1° gennaio per età\"\n", 
    "\n",
    "\"Popolazione per età, vista per territorio - Tutte le regioni\"\n"
]

def chunk_list(lst, n):
    """Divide una lista in sotto-liste di lunghezza n."""
    return [lst[i:i + n] for i in range(0, len(lst), n)]

if not os.path.exists(DESTINATION_PATH):
    os.makedirs(DESTINATION_PATH)

for filename in os.listdir(SOURCE_PATH):
    if not filename.endswith(".csv"):
        continue
    
    lines = []
    with open(SOURCE_PATH + filename, 'r') as f:
        lines = f.readlines()
        lines = [line.replace('Territorio/Età;', 'Codice regione;Regione') for line in lines]
        lines = [line.replace(';', ',') for line in lines]
        lines = [line.replace(',,', ',') for line in lines]
        lines = [line for line in lines if line not in BLACKLIST]
    chunked_list = chunk_list(lines, 70)
    
    for chunck in chunked_list:
        year = int(chunck[0].split(' ')[-1][:-2])
        if "Tutte le cittadinanze" not in chunck[0]:
            continue
        
        chunck = chunck[1:]
        gender_chunk = chunk_list(chunck, 23)
        
        for inner_chunck in gender_chunk:
            inner_chunck = inner_chunck[:-1]
            gender = inner_chunck[1].split(',')[-1].strip()
            del inner_chunck[1]
            if gender not in ["Maschi", "Femmine"]:
                continue
            
            with open(DESTINATION_PATH + f"{year}_{gender[0]}.csv", 'w') as f:
                f.writelines(inner_chunck)

In [12]:
# Lettura dei dati

SOURCE_PATH = DESTINATION_PATH

temp = {
    str(year): {}
    for year in range(2002, 2019 + 1)
}

for filename in os.listdir(SOURCE_PATH):
    if not filename.endswith('.csv'):
        continue
    name = filename.split('.')[0]
    year, gender = name.split('_')
    
    item = {
        gender: pd.read_csv(SOURCE_PATH + filename, encoding='latin1')
    }
    temp[year].update(item)
    
    
datasets = {}
for year, data in temp.items():
    for gender, df in data.items():
        df["Sesso"] = gender 
        data[gender] = df
    datasets[year] = pd.concat(data.values(), ignore_index=True)

In [13]:
# Pulizia dei dati

for k, df in datasets.items():
    if int(k) != 2019:    
        df = df.drop(columns=["Codice regione"])
        datasets[k] = df
        
del datasets["2019"]


In [14]:
# Strutturazione dei dati

COLUMN_NAMES = {
    "Regione": "Area",
    "Sesso": "Sesso",
    "Anno": "Anno"
}

def create_age_bins(df):
        age_bins = {
            '0-9': df[[str(i) for i in range(0, 10)]].sum(axis=1),
            '10-19': df[[str(i) for i in range(10, 20)]].sum(axis=1),
            '20-29': df[[str(i) for i in range(20, 30)]].sum(axis=1),
            '30-39': df[[str(i) for i in range(30, 40)]].sum(axis=1),
            '40-49': df[[str(i) for i in range(40, 50)]].sum(axis=1),
            '50-59': df[[str(i) for i in range(50, 60)]].sum(axis=1),
            '60-69': df[[str(i) for i in range(60, 70)]].sum(axis=1),
            '70-79': df[[str(i) for i in range(70, 80)]].sum(axis=1),
            '80+': df[[str(i) for i in range(80, 101)]].sum(axis=1)  # Età da 80 a 100 e oltre
        }

        # Creiamo un nuovo DataFrame solo con le colonne rilevanti
        df_age_bins = pd.DataFrame(age_bins)
        
        return df_age_bins

for k, df in datasets.items():
    df["Anno"] = int(k)
    
    df_age_bins = create_age_bins(df)
    df_final = pd.concat([df[['Anno', 'Regione', 'Sesso']], df_age_bins], axis=1)
    df_final = df_final.rename(columns=COLUMN_NAMES)
    datasets[k] = df_final

In [15]:
# Aggiunta del territorio nazionale

for k, df in datasets.items():
    df: pd.DataFrame
    years = list(df["Anno"].unique())
    
    for year in years:
        df_year = df[df["Anno"] == year]
        
        # Creazione del territorio nazionale Maschile
        df_year_m = df_year[df_year["Sesso"] == "M"]
        new_item_m = {
            "Area": "Italia",
            "Anno": year,
            "Sesso": "M",
            "0-9": df_year_m["0-9"].sum(),
            "10-19": df_year_m["10-19"].sum(),
            "20-29": df_year_m["20-29"].sum(),
            "30-39": df_year_m["30-39"].sum(),
            "40-49": df_year_m["40-49"].sum(),
            "50-59": df_year_m["50-59"].sum(),
            "60-69": df_year_m["60-69"].sum(),
            "70-79": df_year_m["70-79"].sum(),
            "80+": df_year_m["80+"].sum(),
        }
        new_df_m = pd.DataFrame([new_item_m])
        
        # Creazione del territorio nazionale Femminile
        df_year_f = df_year[df_year["Sesso"] == "F"]
        new_item_f = {
            "Area": "Italia",
            "Anno": year,
            "Sesso": "F",
            "0-9": df_year_f["0-9"].sum(),
            "10-19": df_year_f["10-19"].sum(),
            "20-29": df_year_f["20-29"].sum(),
            "30-39": df_year_f["30-39"].sum(),
            "40-49": df_year_f["40-49"].sum(),
            "50-59": df_year_f["50-59"].sum(),
            "60-69": df_year_f["60-69"].sum(),
            "70-79": df_year_f["70-79"].sum(),
            "80+": df_year_f["80+"].sum(),
        }
        new_item_f = pd.DataFrame([new_item_f])
        
        update_df = pd.concat([df, new_df_m, new_item_f], ignore_index=True)
        datasets[k] = update_df

In [16]:
# Salvataggio dei dati

DESTINATION_PATH = PATH_CLEANED_DATA

if not os.path.exists(DESTINATION_PATH):
    os.makedirs(DESTINATION_PATH)
    
final_dataset = pd.concat(datasets, ignore_index=True)

final_dataset = final_dataset.sort_values(by=["Area", "Anno", "Sesso"])
final_dataset.to_parquet(DESTINATION_PATH + 'popolazione_2002-2018.parquet', index=False)
final_dataset.to_csv(DESTINATION_PATH + 'popolazione_2002-2018.csv', index=False)

### Dataset dagli anni 2019 agli anni 2023

In [17]:
# Lettura dei dati

SOURCE_PATH = PATH_RAW_DATA + '2019-2024/'

datasets = {}

for filename in os.listdir(SOURCE_PATH):
    if filename.endswith('.csv') and filename != '2024.csv':
        year = filename.split('.')[0]
        datasets[year] = pd.read_csv(SOURCE_PATH + filename, skiprows=1, encoding='latin1', sep=';')

In [18]:
# Pulizia dei dati

keep_columns = ["Regione", "ETA", "Totale maschi", "Totale femmine"]

for k, df in datasets.items():
    df = df[df["ETA"] != 999]
    df = df.drop(columns=[col for col in df.columns if col not in keep_columns])
    datasets[k] = df   

In [19]:
# Strutturazione dei dati

COLUMN_NAMES = {
    "Regione": "Area",
}

for k, df in datasets.items():
    df["Anno"] = int(k)
    
    # Suddivisione tra maschi e femmine 
    df_male = df[["Anno", "Regione", "ETA", "Totale maschi"]].copy()
    df_male["Sesso"] = "M"
    df_male.rename(columns={"Totale maschi": "Popolazione"}, inplace=True)
    
    df_female = df[["Anno", "Regione", "ETA", "Totale femmine"]].copy()
    df_female["Sesso"] = "F"
    df_female.rename(columns={"Totale femmine": "Popolazione"}, inplace=True)
    
    df = pd.concat([df_male, df_female], ignore_index=True)
    
    df['Fascia_Eta'] = pd.cut(df['ETA'], bins=AGE_GROUP["age_bins"], labels=AGE_GROUP["age_labels"], right=True, include_lowest=True)

    dataset_pivot = df.pivot_table(index=['Regione', 'Anno', 'Sesso'], 
                                        columns='Fascia_Eta', 
                                        values='Popolazione', 
                                        aggfunc='sum',
                                        observed=True).reset_index()
    
    dataset_pivot.columns.name = None 
    dataset_pivot = dataset_pivot.rename_axis(None, axis=1) 
    dataset_pivot = dataset_pivot.rename(columns=COLUMN_NAMES)
    
    datasets[k] = dataset_pivot

In [20]:
# Aggiunta del terrorio nazionale

for k, df in datasets.items():
    years = list(df["Anno"].unique())
    for year in years:
        df_year = df[df["Anno"] == year]
    
        # Creazione del territorio nazionale Maschile
        df_year_m = df_year[df_year["Sesso"] == "M"]
        new_item_m = {
            "Area": "Italia",
            "Anno": year,
            "Sesso": "M",
            "0-9": df_year_m["0-9"].sum(),
            "10-19": df_year_m["10-19"].sum(),
            "20-29": df_year_m["20-29"].sum(),
            "30-39": df_year_m["30-39"].sum(),
            "40-49": df_year_m["40-49"].sum(),
            "50-59": df_year_m["50-59"].sum(),
            "60-69": df_year_m["60-69"].sum(),
            "70-79": df_year_m["70-79"].sum(),
            "80+": df_year_m["80+"].sum(),
        }
        new_df_m = pd.DataFrame([new_item_m])
        
        # Creazione del territorio nazionale Femminile
        df_year_f = df_year[df_year["Sesso"] == "F"]
        new_item_f = {
            "Area": "Italia",
            "Anno": year,
            "Sesso": "F",
            "0-9": df_year_f["0-9"].sum(),
            "10-19": df_year_f["10-19"].sum(),
            "20-29": df_year_f["20-29"].sum(),
            "30-39": df_year_f["30-39"].sum(),
            "40-49": df_year_f["40-49"].sum(),
            "50-59": df_year_f["50-59"].sum(),
            "60-69": df_year_f["60-69"].sum(),
            "70-79": df_year_f["70-79"].sum(),
            "80+": df_year_f["80+"].sum(),
        }
        new_item_f = pd.DataFrame([new_item_f])
        
        update_df = pd.concat([df, new_df_m, new_item_f], ignore_index=True)
        datasets[k] = update_df
    

In [21]:
# Salvataggio dei dati

DESTINATION_PATH = PATH_CLEANED_DATA

if not os.path.exists(DESTINATION_PATH):
    os.makedirs(DESTINATION_PATH)
    
final_dataset = pd.concat(datasets, ignore_index=True)
final_dataset = final_dataset.sort_values(by=["Area", "Anno", "Sesso"])
final_dataset.to_parquet(DESTINATION_PATH + 'popolazione_2019-2023.parquet', index=False)
final_dataset.to_csv(DESTINATION_PATH + 'popolazione_2019-2023.csv', index=False)

# Feature Engineering

In [26]:
# Caricamento dei dati

SOURCE_PATH = PATH_CLEANED_DATA

datasets = []

for filename in os.listdir(SOURCE_PATH):
    if filename.endswith('.parquet'):
        datasets.append(pd.read_parquet(SOURCE_PATH + filename))
        
dataset = pd.concat(datasets, ignore_index=True)

In [27]:
# [F] Popolazione totale

age_columns = dataset.filter(regex=r'^\d+-\d+$').columns
dataset['Popolazione_Totale'] = dataset[age_columns].sum(axis=1)

In [28]:
# [F] Percentuale di popolazione per fascia d'età

for col in age_columns:
    dataset[col + '_Perc'] = (dataset[col] / dataset['Popolazione_Totale']) * 100

In [29]:
# [F] Crescita della popolazione

dataset = dataset.sort_values(by=['Area', 'Sesso', 'Anno'])

dataset = dataset.fillna(0)
dataset['Crescita_Popolazione_Totale'] = dataset.groupby(['Area', 'Sesso'])['Popolazione_Totale'].pct_change() * 100                                                   
for col in age_columns:
    dataset[f'Crescita_{col}'] = dataset.groupby(['Area', 'Sesso'])[col].pct_change() * 100
                                        
dataset['Crescita_Popolazione_Totale'] = dataset.groupby(['Area', 'Sesso'])['Crescita_Popolazione_Totale'].transform(lambda x: x.fillna(0))
for col in age_columns:
    dataset[f'Crescita_{col}'] = dataset.groupby(['Area', 'Sesso'])[f'Crescita_{col}'].transform(lambda x: x.fillna(0))
    
dataset = dataset.drop_duplicates(subset=['Anno', 'Area', 'Sesso'])

print(dataset.head())

KeyError: Index(['Sex', 'Year', 'Territory'], dtype='object')